In [1]:
import requests
import json
from dateutil import parser
import pandas as pd
from pprint import pprint as original_pprint
import os
from together import Together
import numpy as np

In [3]:
!pip install bm25s

In [4]:
import joblib
import numpy as np
import bm25s
import os
from sentence_transformers import SentenceTransformer

2025-09-21 05:12:32.627201: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-21 05:12:32.789902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758431552.845307   20242 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758431552.862381   20242 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758431552.997639   20242 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
def read_dataframe(path):
    df = pd.read_csv(path)

    # Apply the custom date formatting function to the relevant columns
    df['published_at'] = df['published_at'].apply(format_date)
    df['updated_at'] = df['updated_at'].apply(format_date)

    # Convert the DataFrame to dictionary after formatting
    df= df.to_dict(orient='records')
    return df

In [6]:
def pprint(*args, **kwargs):
    kwargs.setdefault('sort_dicts', False)
    original_pprint(*args, **kwargs)

In [7]:
def generate_with_single_input(prompt: str, 
                               role: str = 'assistant', 
                               top_p: float = 0, 
                               temperature: float = 0,
                               max_tokens: int = 500,
                               model: str ="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                               together_api_key = None,
                              **kwargs):
    
    if top_p is None:
        top_p = 'none'
    if temperature is None:
        temperature = 'none'

    payload = {
            "model": model,
            "messages": [{'role': role, 'content': prompt}],
            "top_p": top_p,
            "temperature": temperature,
            "max_tokens": max_tokens,
            **kwargs
                  }
    if (not together_api_key) and ('TOGETHER_API_KEY' not in os.environ):
        url = os.path.join('https://proxy.dlai.link/coursera_proxy/together', 'v1/chat/completions')   
        response = requests.post(url, json = payload, verify=False)
        if not response.ok:
            raise Exception(f"Error while calling LLM: f{response.text}")
        try:
            json_dict = json.loads(response.text)
        except Exception as e:
            raise Exception(f"Failed to get correct output from LLM call.\nException: {e}\nResponse: {response.text}")
    else:
        if together_api_key is None:
            together_api_key = os.environ['TOGETHER_API_KEY']
        client = Together(api_key =  together_api_key)
        json_dict = client.chat.completions.create(**payload).model_dump()
        json_dict['choices'][-1]['message']['role'] = json_dict['choices'][-1]['message']['role'].name.lower()
    try:
        output_dict = {'role': json_dict['choices'][-1]['message']['role'], 'content': json_dict['choices'][-1]['message']['content']}
    except Exception as e:
        raise Exception(f"Failed to get correct output dict. Please try again. Error: {e}")
    return output_dict

In [8]:
# Distance formulas. 
# In this ungraded lab, distance formulas will be implemented here. However, in future assignments, you will import functions from specialized libraries.
def cosine_similarity(v1, array_of_vectors):
    """
    Compute the cosine similarity between a vector and an array of vectors.
    
    Parameters:
    v1 (array-like): The first vector.
    array_of_vectors (array-like): An array of vectors or a single vector.

    Returns:
    list: A list of cosine similarities between v1 and each vector in array_of_vectors.
    """
    # Ensure that v1 is a numpy array
    v1 = np.array(v1)
    # Initialize a list to store similarities
    similarities = []
    
    # Check if array_of_vectors is a single vector
    if len(np.shape(array_of_vectors)) == 1:
        array_of_vectors = [array_of_vectors]
    
    # Iterate over each vector in the array
    for v2 in array_of_vectors:
        # Convert the current vector to a numpy array
        v2 = np.array(v2)
        # Compute the dot product of v1 and v2
        dot_product = np.dot(v1, v2)
        # Compute the norms of the vectors
        norm_v1 = np.linalg.norm(v1)
        norm_v2 = np.linalg.norm(v2)
        # Compute the cosine similarity and append to the list
        similarity = dot_product / (norm_v1 * norm_v2)
        similarities.append(similarity)
    return np.array(similarities)

In [9]:
import ipywidgets as widgets
from IPython.display import display, Markdown

def display_widget(llm_call_func, semantic_search_retrieve, bm25_retrieve, reciprocal_rank_fusion):
    def on_button_click(b):
        query = query_input.value
        top_k = slider.value
        # Clear existing outputs
        for output in [output1, output2, output3, output4]:
            output.clear_output()
        status_output.clear_output()
        # Display "Generating..." message
        status_output.append_stdout("Generating...\n")
        # Update outputs one by one
        results = [
            (output1, llm_call_func, query, True, top_k, semantic_search_retrieve),
            (output2, llm_call_func, query, True, top_k, bm25_retrieve),
            (output3, llm_call_func, query, True, top_k, reciprocal_rank_fusion),
            (output4, llm_call_func, query, False, top_k, None)
        ]
        for output, func, query, use_rag, top_k, retriever in results:
            response = func(query=query, use_rag=use_rag, top_k=top_k, retrieve_function=retriever)
            with output:
                display(Markdown(response))
        # Clear "Generating..." message
        status_output.clear_output()
        
    query_input = widgets.Text(
        description='',
        placeholder='Type your query here',
        layout=widgets.Layout(width='100%')
    )
    
    slider = widgets.IntSlider(
        value=5,
        min=1,
        max=20,
        step=1,
        description='Top K:',
        style={'description_width': 'initial'}
    )
    
    output_style = {'border': '1px solid #ccc', 'width': '100%'}
    output1 = widgets.Output(layout=output_style)
    output2 = widgets.Output(layout=output_style)
    output3 = widgets.Output(layout=output_style)
    output4 = widgets.Output(layout=output_style)
    status_output = widgets.Output()

    submit_button = widgets.Button(
        description="Get Responses",
        style={'button_color': '#eee', 'font_color': 'black'}
    )
    submit_button.on_click(on_button_click)

    label1 = widgets.Label(value="Semantic Search")
    label2 = widgets.Label(value="BM25 Search")
    label3 = widgets.Label(value="Reciprocal Rank Fusion")
    label4 = widgets.Label(value="Without RAG")

    display(widgets.HTML("""
    <style>
        .custom-output {
            background-color: #f9f9f9;
            color: black;
            border-radius: 5px;
            border: 1px solid #ccc;
        }
        .widget-text, .widget-button {
            background-color: #f0f0f0 !important;
            color: black !important;
            border: 1px solid #ddd !important;
        }
        .widget-output {
            background-color: #f9f9f9 !important;
            color: black !important;
        }
        input[type="text"] {
            background-color: #f0f0f0 !important;
            color: black !important;
            border: 1px solid #ddd !important;
        }
    </style>
    """))
    
    display(query_input, slider, submit_button, status_output)
    
    # Create individual vertical containers for each label and output
    vbox1 = widgets.VBox([label1, output1], layout={'width': '45%'})
    vbox2 = widgets.VBox([label2, output2], layout={'width': '45%'})
    vbox3 = widgets.VBox([label3, output3], layout={'width': '45%'})
    vbox4 = widgets.VBox([label4, output4], layout={'width': '45%'})
    
    # HBoxes to arrange two VBoxes in each row
    hbox_outputs1 = widgets.HBox([vbox1, vbox2], layout={'justify_content': 'space-between'})
    hbox_outputs2 = widgets.HBox([vbox3, vbox4], layout={'justify_content': 'space-between'})

    def style_outputs(*outputs):
        for output in outputs:
            output.layout.margin = '5px'
            output.layout.height = '300px'
            output.layout.padding = '10px'
            output.layout.overflow = 'auto'
            output.add_class("custom-output")
            
    style_outputs(output1, output2, output3, output4)
    
    # Display two rows with two outputs each
    display(hbox_outputs1)
    display(hbox_outputs2)

In [11]:
def format_date(date_string):
    # Parse the input string into a datetime object
    date_object = parser.parse(date_string)
    # Format the date to "YYYY-MM-DD"
    formatted_date = date_object.strftime("%Y-%m-%d")
    return formatted_date

In [12]:
NEWS_DATA = read_dataframe("news_data_dedup.csv")

In [13]:
pprint(NEWS_DATA[5])

{'guid': '18ba9f2676859f393a271d15692a9c6e',
 'title': 'WATCH: Would you pay a tourist fee to enter Venice?',
 'description': 'From Thursday visitors making a trip to the famous city at '
                'peak times will be charged a trial entrance fee.',
 'venue': 'BBC',
 'url': 'https://www.bbc.co.uk/news/world-europe-68898441',
 'published_at': '2024-04-25',
 'updated_at': '2024-04-26'}


In [14]:
def query_news(indices):
    """
    Retrieves elements from a dataset based on specified indices.

    Parameters:
    indices (list of int): A list containing the indices of the desired elements in the dataset.
    dataset (list or sequence): The dataset from which elements are to be retrieved. It should support indexing.

    Returns:
    list: A list of elements from the dataset corresponding to the indices provided in list_of_indices.
    """
     
    output = [NEWS_DATA[index] for index in indices]

    return output

In [15]:
# The corpus used will be the title appended with the description
corpus = [x['title'] + " " + x['description'] for x in NEWS_DATA]

# Instantiate the retriever by passing the corpus data
BM25_RETRIEVER = bm25s.BM25(corpus=corpus)

# Tokenize the chunks
tokenized_data = bm25s.tokenize(corpus)

# Index the tokenized chunks within the retriever
BM25_RETRIEVER.index(tokenized_data)

# Tokenize the same query used in the previous exercise
sample_query = "What are the recent news about GDP?"
tokenized_sample_query = bm25s.tokenize(sample_query)

# Get the retrieved results and their respective scores
results, scores = BM25_RETRIEVER.retrieve(tokenized_sample_query, k=3)

print(f"Results for query: {sample_query}\n")
for doc in results[0]:
  print(f"Document retrieved {corpus.index(doc)} : {doc}\n")

Split strings:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/870 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Results for query: What are the recent news about GDP?

Document retrieved 752 : GDP and the Dow Are Up. But What About American Well-Being? The standard ways of measuring economic growth don’t capture what life is like for real people. A new metric offers a better alternative, especially for seeing disparities across the country.

Document retrieved 673 : What the GDP Report Says About Inflation: A Hot First Quarter Thursday’s gross domestic product report suggests that a widely watched inflation reading due Friday could be worse than expected.

Document retrieved 289 : A GDP Warning as Signs of Stagflation Appear Slower growth and persistent inflation explain why voters feel glum about the economy.



In [16]:
# Use these as a global defined BM25 retriever objects

corpus = [x['title'] + " " + x['description'] for x in NEWS_DATA]
BM25_RETRIEVER = bm25s.BM25(corpus=corpus)
TOKENIZED_DATA = bm25s.tokenize(corpus)
BM25_RETRIEVER.index(TOKENIZED_DATA)

Split strings:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/870 [00:00<?, ?it/s]

In [18]:
# GRADED CELL

def bm25_retrieve(query: str, top_k: int = 5):
    """
    Retrieves the top k relevant documents for a given query using the BM25 algorithm.

    This function tokenizes the input query and uses a pre-indexed BM25 retriever to
    search through a collection of documents. It returns the indices of the top k documents
    that are most relevant to the query.

    Args:
        query (str): The search query for which documents need to be retrieved.
        top_k (int): The number of top relevant documents to retrieve. Default is 5.

    Returns:
        List[int]: A list of indices corresponding to the top k relevant documents
        within the corpus.
    """
    ### START CODE HERE ###

    # Tokenize the query using the 'tokenize' function from the 'bm25s' module
    tokenized_query = bm25s.tokenize(query)
    
    # Use the 'BM25_RETRIEVER' to retrieve documents and their scores based on the tokenized query
    # Retrieve the top 'k' documents
    results, scores = BM25_RETRIEVER.retrieve(tokenized_query, k=top_k)

    # Extract the first element from 'results' to get the list of retrieved documents
    results = results[0]

    # Convert the retrieved documents into their corresponding indices in the results list
    top_k_indices = [corpus.index(result) for result in results]

    ### END CODE HERE ###
    
    return top_k_indices

In [19]:
# Output is a list of indices
bm25_retrieve("What are the recent news about GDP?")

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

[752, 673, 289, 626, 43]

In [21]:
# Load the pre-computed embeddings with joblib
EMBEDDINGS = joblib.load("embeddings.joblib")

In [22]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-base-en-v1.5')

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
# Example usage
query = "RAG is awesome"
# Using, but truncating the result to not pollute the output, don't truncate it in the exercise.
model.encode(query)[:40]

array([ 0.00886308, -0.04775141, -0.00156084,  0.01309998, -0.00206938,
       -0.06157261,  0.01384687,  0.0010149 , -0.04903951, -0.04762556,
       -0.03628187,  0.00478034, -0.0349218 ,  0.05323153,  0.02193961,
        0.03645134,  0.04029364, -0.00453638,  0.01883798, -0.03367381,
        0.02516189, -0.0484363 , -0.04047947,  0.02590901,  0.02175235,
        0.03160365,  0.03937933, -0.03640462, -0.03113292, -0.01247228,
        0.03661649, -0.00458203, -0.00100168, -0.0318879 ,  0.02957138,
        0.01986158, -0.0073747 ,  0.02370171, -0.02151621, -0.07361359],
      dtype=float32)

In [24]:
query1 = "What are the primary colors"
query2 = "Yellow, red and blue"
query3 = "Cats are friendly animals"

query1_embed = model.encode(query1)
query2_embed = model.encode(query2)
query3_embed = model.encode(query3)

print(f"Similarity between '{query1}' and '{query2}' = {cosine_similarity(query1_embed, query2_embed)[0]}")
print(f"Similarity between '{query1}' and '{query3}' = {cosine_similarity(query1_embed, query3_embed)[0]}")

Similarity between 'What are the primary colors' and 'Yellow, red and blue' = 0.7377139329910278
Similarity between 'What are the primary colors' and 'Cats are friendly animals' = 0.4508620500564575


In [25]:
query = "Taylor Swift"
query_embed = model.encode(query)
# The result is a matrix with one matrix per sample. Since there is only one sample (the query), it is a matrix with one matrix within.
# This is why you need to get the first element
similarity_scores = cosine_similarity(query_embed, EMBEDDINGS)
similarity_indices = np.argsort(-similarity_scores) # Sort on decreasing order (sort the negative on increasing order), but return the indices
# Top 2 indices
top_2_indices = similarity_indices[:2]
print(top_2_indices)

[350 176]


In [26]:
# Retrieving the data
query_news(top_2_indices)

[{'guid': '927257674585bb6ef669cf2c2f409fa7',
  'title': '‘The working class can’t afford it’: the shocking truth about the money bands make on tour',
  'description': 'As Taylor Swift tops $1bn in tour revenue, musicians playing smaller venues are facing pitiful fees and frequent losses. Should the state step in to save our live music scene?When you see a band playing to thousands of fans in a sun-drenched festival field, signing a record deal with a major label or playing endlessly from the airwaves, it’s easy to conjure an image of success that comes with some serious cash to boot – particularly when Taylor Swift has broken $1bn in revenue for her current Eras tour. But looks can be deceiving. “I don’t blame the public for seeing a band playing to 2,000 people and thinking they’re minted,” says artist manager Dan Potts. “But the reality is quite different.”Post-Covid there has been significant focus on grassroots music venues as they struggle to stay open. There’s been less focus on

In [27]:
# GRADED CELL 

def semantic_search_retrieve(query, top_k=5):
    """
    Retrieves the top k relevant documents for a given query using semantic search and cosine similarity.

    This function generates an embedding for the input query and compares it against pre-computed document
    embeddings using cosine similarity. The indices of the top k most similar documents are returned.

    Args:
        query (str): The search query for which relevant documents need to be retrieved.
        top_k (int): The number of top relevant documents to retrieve. Default value is 5.

    Returns:
        List[int]: A list of indices corresponding to the top k most relevant documents in the corpus.
    """
    ### START CODE HERE ###
    # Generate the embedding for the query using the pre-trained model
    query_embedding = model.encode(query)
    
    # Calculate the cosine similarity scores between the query embedding and the pre-computed document embeddings
    similarity_scores = cosine_similarity(query_embedding, EMBEDDINGS)
    
    # Sort the similarity scores in descending order and get the indices
    similarity_indices = np.argsort(-similarity_scores)

    # Select the indices of the top k documents as a numpy array
    top_k_indices_array = similarity_indices[:top_k]

    ### END CODE HERE ###
    
    # Cast them to int 
    top_k_indices = [int(x) for x in top_k_indices_array]
    
    return top_k_indices

In [28]:
# Let's see an example
semantic_search_retrieve("What are the recent news about GDP?")

[743, 673, 626, 752, 326]

In [29]:
# GRADED CELL 
def reciprocal_rank_fusion(list1, list2, top_k=5, K=60):
    """
    Fuse rank from multiple IR systems using Reciprocal Rank Fusion.

    Args:
        list1 (list[int]): A list of indices of the top-k documents that match the query.
        list2 (list[int]): Another list of indices of the top-k documents that match the query.
        top_k (int): The number of top documents to consider from each list for fusion. Defaults to 5.
        K (int): A constant used in the RRF formula. Defaults to 60.

    Returns:
        list[int]: A list of indices of the top-k documents sorted by their RRF scores.
    """

    ### START CODE HERE ###

    # Create a dictionary to store the RRF scores for each document index
    rrf_scores = {}

    # Iterate over each document list
    for lst in [list1, list2]:
        # Calculate the RRF score for each document index
        for rank, item in enumerate(lst, start=1): # Start = 1 set the first element as 1 and not 0. 
                                                   # This is a convention on how ranks work (the first element in ranking is denoted by 1 and not 0 as in lists)
            # If the item is not in the dictionary, initialize its score to 0
            if item not in rrf_scores:
                rrf_scores[item] = 0
            # Update the RRF score for each document index using the formula 1 / (rank + K)
            rrf_scores[item] += (1 / (rank + K))

    # Sort the document indices based on their RRF scores in descending order
    sorted_items = sorted(rrf_scores, key=rrf_scores.get, reverse = True)

    # Slice the list to get the top-k document indices
    top_k_indices = [int(x) for x in sorted_items[:top_k]]

    ### END CODE HERE ###

    return top_k_indices

In [30]:
list1 = semantic_search_retrieve('What are the recent news about GDP?')
list2 = bm25_retrieve('What are the recent news about GDP?')
rrf_list = reciprocal_rank_fusion(list1, list2)
print(f"Semantic Search List: {list1}")
print(f"BM25 List: {list2}")
print(f"RRF List: {rrf_list}")

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Semantic Search List: [743, 673, 626, 752, 326]
BM25 List: [752, 673, 289, 626, 43]
RRF List: [673, 752, 626, 743, 289]


In [31]:
def generate_final_prompt(query, top_k, retrieve_function = None, use_rag=True):
    """
    Generates an augmented prompt for a Retrieval-Augmented Generation (RAG) system by retrieving the top_k most 
    relevant documents based on a given query.

    Parameters:
    query (str): The search query for which the relevant documents are to be retrieved.
    top_k (int): The number of top relevant documents to retrieve.
    retrieve_function (callable): The function used to retrieve relevant documents. If 'reciprocal_rank_fusion', 
                                  it will combine results from different retrieval functions.
    use_rag (bool): A flag to determine whether to incorporate retrieved data into the prompt (default is True).

    Returns:
    str: A prompt that includes the top_k relevant documents formatted for use in a RAG system.
    """

    # Define the prompt as the initial query
    prompt = query
    
    # If not using rag, return the prompt
    if not use_rag:
        return prompt


    # Determine which retrieve function to use based on its name.
    if retrieve_function.__name__ == 'reciprocal_rank_fusion':
        # Retrieve top documents using two different methods.
        list1 = semantic_search_retrieve(query, top_k)
        list2 = bm25_retrieve(query, top_k)
        # Combine the results using reciprocal rank fusion.
        top_k_indices = retrieve_function(list1, list2, top_k)
    else:
        # Use the provided retrieval function.
        top_k_indices = retrieve_function(query=query, top_k=top_k)
    
    
    # Retrieve documents from the dataset using the indices.
    relevant_documents = query_news(top_k_indices)
    
    formatted_documents = []

    # Iterate over each retrieved document.
    for document in relevant_documents:
        # Format each document into a structured string.
        formatted_document = (
            f"Title: {document['title']}, Description: {document['description']}, "
            f"Published at: {document['published_at']}\nURL: {document['url']}"
        )
        # Append the formatted string to the main data string with a newline for separation.
        formatted_documents.append(formatted_document)

    retrieve_data_formatted = "\n".join(formatted_documents)
    
    prompt = (
        f"Answer the user query below. There will be provided additional information for you to compose your answer. "
        f"The relevant information provided is from 2024 and it should be added as your overall knowledge to answer the query, "
        f"you should not rely only on this information to answer the query, but add it to your overall knowledge."
        f"Query: {query}\n"
        f"2024 News: {retrieve_data_formatted}"
    )

    
    return prompt

In [ ]:
os.environ['TOGETHER_API_KEY'] = ""

In [38]:
def generate_with_single_input(prompt: str, 
                               role: str = 'assistant', 
                               top_p: float = 0, 
                               temperature: float = 0,
                               max_tokens: int = 500,
                               model: str ="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                               together_api_key = None,
                              **kwargs):
    
    if top_p is None:
        top_p = 'none'
    if temperature is None:
        temperature = 'none'

    payload = {
            "model": model,
            "messages": [{'role': role, 'content': prompt}],
            "top_p": top_p,
            "temperature": temperature,
            "max_tokens": max_tokens,
            **kwargs
                  }
    if (not together_api_key) and ('TOGETHER_API_KEY' not in os.environ):
        url = os.path.join('https://proxy.dlai.link/coursera_proxy/together', 'v1/chat/completions')   
        response = requests.post(url, json = payload, verify=False)
        if not response.ok:
            raise Exception(f"Error while calling LLM: f{response.text}")
        try:
            json_dict = json.loads(response.text)
        except Exception as e:
            raise Exception(f"Failed to get correct output from LLM call.\nException: {e}\nResponse: {response.text}")
    else:
        if together_api_key is None:
            together_api_key = os.environ['TOGETHER_API_KEY']
        client = Together(api_key =  together_api_key)
        json_dict = client.chat.completions.create(**payload).model_dump()
        json_dict['choices'][-1]['message']['role'] = json_dict['choices'][-1]['message']['role'].name.lower()
    try:
        output_dict = {'role': json_dict['choices'][-1]['message']['role'], 'content': json_dict['choices'][-1]['message']['content']}
    except Exception as e:
        raise Exception(f"Failed to get correct output dict. Please try again. Error: {e}")
    return output_dict

In [39]:
def llm_call(query, retrieve_function = None, top_k = 5,use_rag = True):

    # Get the system and user dictionaries
    prompt = generate_final_prompt(query, top_k = top_k, retrieve_function = retrieve_function, use_rag = use_rag)

    generated_response = generate_with_single_input(prompt)

    generated_message = generated_response['content']
    
    return generated_message

In [40]:
query = "Recent news in technology. Provide sources."
print(llm_call(query, retrieve_function = semantic_search_retrieve))

There have been several recent news in the technology sector. Some of the key developments include:

1. **Artificial Intelligence Sparks 'Game of Thrones' in the Chip Industry**: The rapid advancement of artificial intelligence (AI) is transforming the semiconductor industry, creating new winners and losers. The supply chain is becoming increasingly complex, with various players vying for dominance. (Source: El Pais, April 12, 2024)

2. **Tech Spending Still Proves Thorny for Some Advertising Companies**: The technology sector's slower pace of business has continued to affect some advertising holding companies in the first quarter. However, there are signs that things might be looking up. (Source: The Wall Street Journal, April 26, 2024)

3. **Market Talks: T-Mobile, Imax, Rogers Communications, and More**: The latest Market Talks covering Technology, Media, and Telecom have highlighted key developments in the sector, including updates on T-Mobile, Imax, and Rogers Communications. (Sou

In [ ]:
display_widget(llm_call, semantic_search_retrieve, bm25_retrieve, reciprocal_rank_fusion)

HTML(value='\n    <style>\n        .custom-output {\n            background-color: #f9f9f9;\n            color…

Text(value='', layout=Layout(width='100%'), placeholder='Type your query here')

IntSlider(value=5, description='Top K:', max=20, min=1, style=SliderStyle(description_width='initial'))

Button(description='Get Responses', style=ButtonStyle(button_color='#eee'))

Output()